In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from mpl_toolkits.mplot3d import Axes3D

NCHANNELS = 64
#all distances in millimeters, times in nanoseconds
ZCELL = 13
XCELL = 42
VDRIFT = .054
Z_SEP = 784

max_slope = float(np.sqrt(3)) #maximum acceptable angle from the vertical

def allowed_slope(xs,ys):
#checks if the slope between the start and end point is greater than the maximum allowed value
    x = sorted(xs)
    y = sorted(ys)
    if np.around(y[-1]-y[0]) == 0:
        slope = 100
    else:
        slope = float((x[-1]-x[0])/(y[-1]-y[0]))
    if slope > max_slope:
        return False
    else:
        return True

# function to try possible combinations of points and select the one with the best line of fit
def find_fit(df):
    chambs = df.groupby('y')
    list1 = []
    for i,chamb in chambs:
        list1.append(chamb.to_numpy())
    points = np.array(list(itertools.product(*list1)))

    #iterate over all possible combinations of points and return the fit with best chi squared
    chisq_best = 20. #maximum acceptable chi squared
    dof = 2 #degrees of freedom in the fit
    fit_best = []
    x_best = []
    y_best = []
    count = np.arange(len(points))
    for i in count:
        xs = [x[0] for x in points[i]]
        ys = [x[1] for x in points[i]]
        fit, chisq, _, _, _ = np.polyfit(ys,xs,1,full = True)
        x_fit = list(np.poly1d(fit)(ys))
        
        #ignore combinations with slopes that aren't allowed
        if allowed_slope(x_fit,ys) == True:
            #handles a combination with chisq << 1
            if chisq.size == 0:
                chisq_best = 0
                fit_best = fit
                x_best = xs
                y_best = ys 
            elif float(chisq)/dof < chisq_best:
                chisq_best = chisq/dof
                fit_best = fit
                x_best = xs
                y_best = ys
    
    fit_pts = list(np.poly1d(fit_best)(y_best))
    return x_best,y_best,fit_pts,float(chisq_best)
    
    
def removezeros(arr):
    zeros = np.all(np.equal(arr, 0), axis=1)
    arr = arr[~zeros]
    return arr

#creates local reconstructions from the output of processhits.py
def local_reconstruction_xleft_xright(path,start = 0,end = None):
    #start and end specify how many events to try and locally reconstruct
    df= pd.DataFrame(columns = np.arange(20))
    rej_count = 0
    event = 0
    chis = []
    badchisq = 0
    with open(path) as fp:
        if end == None:
            for i, line in enumerate(fp):
            #convert each line of text file (ie. event) to a readable list of the relevant data
                event +=1
                data = line.split(' ')
                data = data[2:]
                n = len(data)
                index = 0
                j0 = j1 = j2 = j3 = 0
                pts0 = np.zeros((2*n,2))
                pts1 = np.zeros((2*n,2))
                pts2 = np.zeros((2*n,2))
                pts3 = np.zeros((2*n,2))

                while index < n:
                    layer = int(data[index+1])
                    conditions  = [(layer == 1 ),
                                  (layer == 3 ),
                                  (layer == 2 ),
                                  (layer == 4 ),
                                  ]
                    pos_z       = [  ZCELL*3.5,    ZCELL*1.5,    ZCELL*2.5,    ZCELL*0.5, ]
                    if float(data[index]) == 0:
                        pts0[j0,0] = float(data[index+2])
                        pts0[j0+1,0] = float(data[index+3])
                        pts0[j0,1] = pts0[j0+1,1] = np.select(conditions, pos_z,default=0).astype(np.float16)
                        index +=5
                        j0 += 2
                    elif float(data[index]) == 1.:
                        pts1[j1,0] = float(data[index+2])
                        pts1[j1+1,0] = float(data[index+3])
                        pts1[j1,1] = pts1[j1+1,1] = np.select(conditions, pos_z,default=1).astype(np.float16)
                        index +=5
                        j1 += 2
                    elif float(data[index]) == 2.:
                        pts2[j2,0] = float(data[index+2])
                        pts2[j2+1,0] = float(data[index+3])
                        pts2[j2,1] = pts2[j2+1,1] = np.select(conditions, pos_z,default=2).astype(np.float16)
                        index +=5
                        j2 += 2
                    else:
                        pts3[j3,0] = float(data[index+2])
                        pts3[j3+1,0] = float(data[index+3])
                        pts3[j3,1] = pts3[j3+1,1] = np.select(conditions, pos_z,default=3).astype(np.float16)
                        index +=5
                        j3 += 2

                pts0 = pd.DataFrame(removezeros(pts0),columns = ('x','y'))
                pts1 = pd.DataFrame(removezeros(pts1),columns = ('x','y'))
                pts2 = pd.DataFrame(removezeros(pts2),columns = ('x','y'))
                pts3 = pd.DataFrame(removezeros(pts3),columns = ('x','y'))

                #filter out events where a chamber didn't have enough hits for a reconstruction
                if len(pts0) == 0 or len(pts1) == 0 or len(pts2) == 0 or len(pts3) == 0:
                    #print('Invalid event: One or more chambers had no hits')
                    rej_count += 1
                    continue

                #return a local reconstruction for each chamber
                x0,z0,fit0,chi0 = find_fit(pts0)
                x1,z1,fit1,chi1 = find_fit(pts1)
                x2,z2,fit2,chi2 = find_fit(pts2)
                x3,z3,fit3,chi3 = find_fit(pts3)
        
                #filter out events with fits below the chi squared threshold
                if len(x0) == 0 or len(x1) == 0 or len(x2) == 0 or len(x3) == 0:
                    rej_count += 1
                    badchisq += 1
                else:
                    #if all chambers had acceptable local reconstructions, plot them
                    chis.extend([chi0,chi1,chi2,chi3])   
                    ch1 = x0+z0
                    ch2 = x1+list(np.asarray(z1)+4*ZCELL)
                    ch3 = x2+list(np.asarray(z2)+Z_SEP)
                    ch4 = x3+list(np.asarray(z3)+4*ZCELL+Z_SEP)
                    #add the event number to each set of coordinates to keep track of which event is which
                    ch1.insert(0,event)
                    ch2.insert(0,event)
                    ch3.insert(0,event)
                    ch4.insert(0,event)
                    chambs = [ch1,ch2,ch3,ch4]

                    for i in np.arange(len(chambs)):
                        df2 = pd.DataFrame(chambs[i],dtype = float)
                        df = df.append(df2.T,ignore_index = True)
        else:
            for i, line in enumerate(fp):
                if start <= i < end:
                #convert each line of text file (ie. event) to a readable list of the relevant data
                    event +=1
                    data = line.split(' ')
                    data = data[2:]
                    n = len(data)
                    index = 0
                    j0 = j1 = j2 = j3 = 0
                    pts0 = np.zeros((2*n,2))
                    pts1 = np.zeros((2*n,2))
                    pts2 = np.zeros((2*n,2))
                    pts3 = np.zeros((2*n,2))
                    
                    while index < n:
                        layer = int(data[index+1])
                        conditions  = [(layer == 1 ),
                                      (layer == 3 ),
                                      (layer == 2 ),
                                      (layer == 4 ),
                                      ]
                        pos_z       = [  ZCELL*3.5,    ZCELL*1.5,    ZCELL*2.5,    ZCELL*0.5, ]
                        if float(data[index]) == 0:
                            pts0[j0,0] = float(data[index+2])
                            pts0[j0+1,0] = float(data[index+3])
                            pts0[j0,1] = pts0[j0+1,1] = np.select(conditions, pos_z,default=0).astype(np.float16)
                            index +=5
                            j0 += 2
                        elif float(data[index]) == 1.:
                            pts1[j1,0] = float(data[index+2])
                            pts1[j1+1,0] = float(data[index+3])
                            pts1[j1,1] = pts1[j1+1,1] = np.select(conditions, pos_z,default=0).astype(np.float16)
                            index +=5
                            j1 += 2
                        elif float(data[index]) == 2.:
                            pts2[j2,0] = float(data[index+2])
                            pts2[j2+1,0] = float(data[index+3])
                            pts2[j2,1] = pts2[j2+1,1] = np.select(conditions, pos_z,default=0).astype(np.float16)
                            index +=5
                            j2 += 2
                        else:
                            pts3[j3,0] = float(data[index+2])
                            pts3[j3+1,0] = float(data[index+3])
                            pts3[j3,1] = pts3[j3+1,1] = np.select(conditions, pos_z,default=0).astype(np.float16)
                            index +=5
                            j3 += 2

                    pts0 = pd.DataFrame(removezeros(pts0),columns = ('x','y'))
                    pts1 = pd.DataFrame(removezeros(pts1),columns = ('x','y'))
                    pts2 = pd.DataFrame(removezeros(pts2),columns = ('x','y'))
                    pts3 = pd.DataFrame(removezeros(pts3),columns = ('x','y'))

                    #filter out events where a chamber didn't have enough hits for a reconstruction
                    if len(pts0) == 0 or len(pts1) == 0 or len(pts2) == 0 or len(pts3) == 0:
                        #print('Invalid event: One or more chambers had no hits')
                        rej_count += 1
                        continue

                    #return a local reconstruction for each chamber
                    x0,z0,fit0,chi0 = find_fit(pts0)
                    x1,z1,fit1,chi1 = find_fit(pts1)
                    x2,z2,fit2,chi2 = find_fit(pts2)
                    x3,z3,fit3,chi3 = find_fit(pts3)

                    #filter out events with fits below the chi squared threshold
                    if len(x0) == 0 or len(x1) == 0 or len(x2) == 0 or len(x3) == 0:
                        rej_count += 1
                        badchisq += 1
                    else:
                        #if all chambers had acceptable local reconstructions,plot them
                        chis.extend([chi0,chi1,chi2,chi3])
                        fig1,axes = plt.subplots(nrows = 2,ncols = 2,figsize =(8,8),constrained_layout=True)
                        axes[0,0].plot(fit0,z0)
                        axes[0,0].scatter(x0,z0,color = 'black',marker = '.')
                        axes[0,0].set_title('Event '+str(event)+' Chamber 1')
                        axes[0,0].set_xlim(min(x0)-5,max(x0)+5)
                        axes[0,0].set_ylim(0, 52)
                        axes[0,1].plot(fit1,z1)
                        axes[0,1].scatter(x1,z1,color = 'black',marker = '.')
                        axes[0,1].set_title('Event '+str(event)+' Chamber 2')
                        axes[0,1].set_xlim(min(x1)-5,max(x1)+5)
                        axes[0,1].set_ylim(0, 52)
                        axes[1,0].plot(fit2,z2)
                        axes[1,0].scatter(x2,z2,color = 'black',marker = '.')
                        axes[1,0].set_title('Event '+str(event)+' Chamber 3')
                        axes[1,0].set_xlim(min(x2)-5,max(x2)+5)
                        axes[1,0].set_ylim(0, 52)
                        axes[1,1].plot(fit3,z3)
                        axes[1,1].scatter(x3,z3,color = 'black',marker = '.')
                        axes[1,1].set_title('Event '+str(event)+' Chamber 4')
                        axes[1,1].set_xlim(min(x3)-5,max(x3)+5)
                        axes[1,1].set_ylim(0, 52)
                        ch1 = x0+z0
                        ch2 = x1+list(np.asarray(z1)+4*ZCELL)
                        ch3 = x2+list(np.asarray(z2)+Z_SEP)
                        ch4 = x3+list(np.asarray(z3)+4*ZCELL+Z_SEP)
                        
                        #add the event number to each set of coordinates to keep track of which event is which
                        ch1.insert(0,event)
                        ch2.insert(0,event)
                        ch3.insert(0,event)
                        ch4.insert(0,event)
                        chambs = [ch1,ch2,ch3,ch4]

                        for i in np.arange(len(chambs)):
                            df2 = pd.DataFrame(chambs[i],dtype = float)
                            df = df.append(df2.T,ignore_index = True)
                            
    accepted = event-rej_count
    print('Events With Acceptable Local Reconstructions: '+str(accepted)+' out of '+str(event))
    fp.close()
    print('Events with One or More Chambers Above the Chi Squared Threshold: '+str(badchisq))
    return df,chis

def total_reconstruction(df,start = 0,end = None,split = False):
#start and end points specift which ones of the accepted events to plot
#set split to True for each event on a separate plot,otherwise they all appear together
    count = len(df.index)
    accepted = 0
    if split:
    #plot each reconstruction separately
        if end == None:
            end = count
        j = 0
        chis3d = []
        chis = []
        while j < count:
            if start*4 <= j < end*4:
                #iterate through the dataframe, plotting one line per event
                ch1 = list(df.loc[j,:].dropna())
                ch2 = list(df.loc[j+1,:].dropna())
                ch3 = list(df.loc[j+2,:].dropna())
                ch4 = list(df.loc[j+3,:].dropna())
                x1 = ch1[1:len(ch1)//2+1]
                z1 = ch1[len(ch1)//2+1:]
                y2 = ch2[1:len(ch2)//2+1]
                z2 = ch2[len(ch2)//2+1:]
                xfit1 = np.polyfit(z1,x1,1)
                yfit1 = np.polyfit(z2,y2,1)
                y1 = list(np.poly1d(yfit1)(z1))
                x2 = list(np.poly1d(xfit1)(z2))

                x3 = ch3[1:len(ch3)//2+1]
                z3 = ch3[len(ch3)//2+1:]
                y4 = ch4[1:len(ch4)//2+1]
                z4 = ch4[len(ch4)//2+1:]
                xfit2 = np.polyfit(z3,x3,1)
                yfit2 = np.polyfit(z4,y4,1)
                y3 = list(np.poly1d(yfit2)(z3))
                x4 = list(np.poly1d(xfit2)(z4))
                event_nr = int(ch1[0])

                #discard if there is not good agreement between corresponding chambers
                x1new = x1+x3
                z1new = z1+z3
                y1new,chisq1,_,_,_ = np.polyfit(x1new,z1new,1,full = True)
                y2new = y2+y4
                z2new = z2+z4
                x2new,chisq2,_,_,_ = np.polyfit(y2new,z2new,1,full = True)
                chis.extend([float(chisq1/2),float(chisq2/2)])

                if float(chisq1/2) < 200. and float(chisq2/2) < 200.:
                    x1.extend(x2)
                    x1.extend(x3)
                    x1.extend(x4)
                    y1.extend(y2)
                    y1.extend(y3)
                    y1.extend(y4)
                    z1.extend(z2)
                    z1.extend(z3)
                    z1.extend(z4)

                    #find a plane of best fit for the x-z axis and y-z axis
                    A_xz = np.vstack((x1, np.ones(len(x1)))).T
                    m_xz,chix,_,_ = np.linalg.lstsq(A_xz, z1,rcond=None)
                    A_yz = np.vstack((y1, np.ones(len(y1)))).T
                    m_yz,chiy,_,_ = np.linalg.lstsq(A_yz, z1,rcond=None)
                    chis3d.extend([float(chix/3),float(chiy/3)])

                    if chix/3 < 500. and chiy/3 < 500.:
                        accepted += 1
                        #calculate points along the intersection of the planes to get best global fit
                        z_final = np.linspace(0,888)
                        x_final = (z_final - m_xz[1])/m_xz[0]
                        y_final = (z_final - m_yz[1])/m_yz[0]
                        label = 'Event '+str(event_nr)
                        fig = plt.figure(figsize =(6,6))
                        ax = Axes3D(fig)
                        ax.set_xlabel("X")
                        ax.set_ylabel("Y")
                        ax.set_zlabel("Z")
                        ax.scatter(x1,y1,z1)
                        ax.plot(x_final,y_final,z_final,label = label)
                        ax.set_title(label)
                        ax.set_xlim(0, 693)
                        ax.set_ylim(0, 693)
                        ax.set_proj_type('ortho')
                        plt.show()
            j += 4
    else:
    #plot all reconstructions together
        fig = plt.figure(figsize =(8,8))
        ax = Axes3D(fig)
        ax.set_xlabel("X")
        ax.set_ylabel("Y")
        ax.set_zlabel("Z")
        if end == None:
            end = count
        j = 0
        chis3d = []
        chis = []
        while j < count:
            if start*4 <= j < end*4:
                #iterate through the dataframe, plotting one line per event
                ch1 = list(df.loc[j,:].dropna())
                ch2 = list(df.loc[j+1,:].dropna())
                ch3 = list(df.loc[j+2,:].dropna())
                ch4 = list(df.loc[j+3,:].dropna())
                x1 = ch1[1:len(ch1)//2+1]
                z1 = ch1[len(ch1)//2+1:]
                y2 = ch2[1:len(ch2)//2+1]
                z2 = ch2[len(ch2)//2+1:]
                xfit1 = np.polyfit(z1,x1,1)
                yfit1 = np.polyfit(z2,y2,1)
                y1 = list(np.poly1d(yfit1)(z1))
                x2 = list(np.poly1d(xfit1)(z2))

                x3 = ch3[1:len(ch3)//2+1]
                z3 = ch3[len(ch3)//2+1:]
                y4 = ch4[1:len(ch4)//2+1]
                z4 = ch4[len(ch4)//2+1:]
                xfit2 = np.polyfit(z3,x3,1)
                yfit2 = np.polyfit(z4,y4,1)
                y3 = list(np.poly1d(yfit2)(z3))
                x4 = list(np.poly1d(xfit2)(z4))
                event_nr = int(ch1[0])

                x1new = x1+x3
                z1new = z1+z3
                y1new,chisq1,_,_,_ = np.polyfit(x1new,z1new,1,full = True)
                y2new = y2+y4
                z2new = z2+z4
                x2new,chisq2,_,_,_ = np.polyfit(y2new,z2new,1,full = True)
                chis.extend([float(chisq1/2),float(chisq2/2)])

                #if limits are provided,only plot the specified range of events
                if float(chisq1/2) < 200. and float(chisq2/2) < 200.:
                    x1.extend(x2)
                    x1.extend(x3)
                    x1.extend(x4)
                    y1.extend(y2)
                    y1.extend(y3)
                    y1.extend(y4)
                    z1.extend(z2)
                    z1.extend(z3)
                    z1.extend(z4)

                    #find a plane of best fit for the x-z axis and y-z axis
                    A_xz = np.vstack((x1, np.ones(len(x1)))).T
                    m_xz,chix,_,_ = np.linalg.lstsq(A_xz, z1,rcond=None)
                    A_yz = np.vstack((y1, np.ones(len(y1)))).T
                    m_yz,chiy,_,_ = np.linalg.lstsq(A_yz, z1,rcond=None)
                    chis3d.extend([float(chix/3),float(chiy/3)])

                    #calculate points along the intersection of the planes to get best global fit
                    if chix/3 < 500. and chiy/3 < 500.:
                        accepted += 1
                        z_final = np.linspace(0,888)
                        x_final = (z_final - m_xz[1])/m_xz[0]
                        y_final = (z_final - m_yz[1])/m_yz[0]
                        label = 'Event '+str(event_nr)
                        ax.scatter(x1,y1,z1)
                        ax.plot(x_final,y_final,z_final,label = label)
                        ax.legend(loc = 'upper left')
                        ax.set_xlim(0, 693)
                        ax.set_ylim(0, 693)
                        plt.show()

            j+=4
    print('Events With Acceptable Global Reconstruction: '+str(accepted)+' out of '+str(count//4))
    return chis,chis3d

In [ ]:
#delete this if not using jupyter notebook
%matplotlib notebook

#use chi_local,chi_2d and chi_global for performance analysis and to perhaps determine different cutoffs
#you must set an end to local_reconstruction_xleft_xright to display local reconstruction plots
data,chi_local = local_reconstruction_xleft_xright('/home/aidan/test files/data_000002_t0.txt')

#recommended to set a start/end to total_reconstruction as well so you don't get too many plots at once
#set split = True for each 3d track to be on a separate figure, otherwise they all appear together
chi_2d,chi_global = total_reconstruction(data)